# Imports

In [10]:
# Standard library imports
import os
import math
from pathlib import Path

# Third-party imports
import pandas as pd
from lightning import pytorch as pl
import torch
from chemprop import data, featurizers, models, nn
from chembl_webresource_client.new_client import new_client

# Local imports

# CUDA
print(f"CUDA available: {torch.cuda.is_available()}")
os.environ['CUDA_VISIBLE_DEVICES'] = '7'

CUDA available: True


# ChEMBL Database

In [24]:
cur_dir = os.path.dirname(os.path.realpath('__file__'))

In [11]:
available_resources = [resource for resource in dir(new_client) if not resource.startswith('_')]
print(available_resources)

['activity', 'activity_supplementary_data_by_activity', 'assay', 'assay_class', 'atc_class', 'binding_site', 'biotherapeutic', 'cell_line', 'chembl_id_lookup', 'chembl_release', 'compound_record', 'compound_structural_alert', 'description', 'document', 'document_similarity', 'drug', 'drug_indication', 'drug_warning', 'go_slim', 'image', 'mechanism', 'metabolism', 'molecule', 'molecule_form', 'official', 'organism', 'protein_classification', 'similarity', 'source', 'substructure', 'target', 'target_component', 'target_relation', 'tissue', 'xref_source']


In [26]:
# Cytochrome P450 3A4 IDs
chembl_ids_cyp3a4 = ['CHEMBL340', 'CHEMBL2111472', 'CHEMBL2364675', 'CHEMBL4523986']

# Initialize the lookup client & fetch
lookup = new_client.chembl_id_lookup
lookup_data = [lookup.get(chembl_id) for chembl_id in chembl_ids_cyp3a4]

# Collect all data with those IDs
target = new_client.target
target_data = [target.get(chembl_id) for chembl_id in chembl_ids_cyp3a4]
df = pd.DataFrame(target_data)
print(df.head())

                                    cross_references      organism  \
0  [{'xref_id': 'P08684', 'xref_name': None, 'xre...  Homo sapiens   
1                                                 []  Homo sapiens   
2                                                 []  Homo sapiens   
3                                                 []  Homo sapiens   

                 pref_name  species_group_flag target_chembl_id  \
0      Cytochrome P450 3A4               False        CHEMBL340   
1  Cytochrome P450 3A4/3A5               False    CHEMBL2111472   
2       Cytochrome P450 3A               False    CHEMBL2364675   
3          Cytochrome P450               False    CHEMBL4523986   

                                   target_components     target_type  tax_id  
0  [{'accession': 'P08684', 'component_descriptio...  SINGLE PROTEIN    9606  
1  [{'accession': 'P08684', 'component_descriptio...  PROTEIN FAMILY    9606  
2  [{'accession': 'P08684', 'component_descriptio...  PROTEIN FAMILY    9606

In [28]:
# Get bioactivities
activities = new_client.activity
activities_data = activities.filter(
    target_chembl_id=chembl_id for chembe
).only(['activity_id', 'assay_type', 'standard_type', 'standard_value', 'standard_units', 'canonical_smiles'])
df = pd.DataFrame(activities_data)
print(df.head())

RetryError: HTTPSConnectionPool(host='www.ebi.ac.uk', port=443): Max retries exceeded with url: /chembl/api/data/activity/CHEMBL340 (Caused by ResponseError('too many 404 error responses'))